<a href="https://colab.research.google.com/github/Mkr-mayank/CNN/blob/main/Cifar100_Using_Efficientnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

In [ ]:
input_shape = (456, 456, 3)
x_train_resized = tf.image.resize(x_train, (456, 456))
x_test_resized = tf.image.resize(x_test, (456, 456))

In [ ]:
x_train_resized = x_train_resized / 255.0
x_test_resized = x_test_resized / 255.0

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

datagen.fit(x_train_resized)

In [ ]:
base_model = EfficientNetB5(include_top=False, weights='imagenet', input_shape=input_shape)

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(100, activation='softmax'))

In [ ]:
base_model.trainable = False

In [ ]:
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [1]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

NameError: name 'EarlyStopping' is not defined

In [2]:
history = model.fit(datagen.flow(x_train_resized, y_train, batch_size=32),
                    validation_data=(x_test_resized, y_test),
                    epochs=100,
                    callbacks=[early_stopping, reduce_lr])

NameError: name 'model' is not defined

In [3]:
for layer in base_model.layers[-20:]:
    layer.trainable = True

NameError: name 'base_model' is not defined

In [4]:
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

NameError: name 'model' is not defined

In [5]:
history_fine = model.fit(datagen.flow(x_train_resized, y_train, batch_size=32),
                         validation_data=(x_test_resized, y_test),
                         epochs=50,
                         callbacks=[early_stopping, reduce_lr])

NameError: name 'model' is not defined

In [ ]:
test_loss, test_acc = model.evaluate(x_test_resized, y_test)
print(f"Test accuracy: {test_acc:.2f}")